In [1]:
import pandas as pd
import re

In [2]:
file_path = "data/FAOSTAT_data_en_9-14-2025.csv"
df_raw = pd.read_csv(file_path)
df_raw.head()

,Domain Code,Domain,Area Code (M49),Country,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Production Tonnes,Flag,Flag Description,Note,Forest_area_pct
0,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2013,2013,t,1040835.0,A,Official figure,NaN,54.442462
1,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2014,2014,t,1109586.0,A,Official figure,NaN,54.321123
2,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2015,2015,t,1275000.0,A,Official figure,NaN,54.199784
3,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2016,2016,t,1146000.0,A,Official figure,NaN,54.041902
4,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2017,2017,t,1627552.0,A,Official figure,NaN,53.843867


In [3]:
df_raw.columns.tolist()

['Domain Code',
 'Domain',
 'Area Code (M49)',
 'Country',
 'Element Code',
 'Element',
 'Item Code (CPC)',
 'Item',
 'Year Code',
 'Year',
 'Unit',
 'Production Tonnes',
 'Flag',
 'Flag Description',
 'Note',
 'Forest_area_pct']

In [4]:
rename_map = {}
if "Area" in df_raw.columns: rename_map["Area"] = "Country"
if "Year" in df_raw.columns: rename_map["Year"] = "Year"
if "Value" in df_raw.columns: rename_map["Value"] = "Production Tonnes"

df_raw = df_raw.rename(columns = rename_map)
df_raw.head()

,Domain Code,Domain,Area Code (M49),Country,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Production Tonnes,Flag,Flag Description,Note,Forest_area_pct
0,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2013,2013,t,1040835.0,A,Official figure,NaN,54.442462
1,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2014,2014,t,1109586.0,A,Official figure,NaN,54.321123
2,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2015,2015,t,1275000.0,A,Official figure,NaN,54.199784
3,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2016,2016,t,1146000.0,A,Official figure,NaN,54.041902
4,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2017,2017,t,1627552.0,A,Official figure,NaN,53.843867


In [5]:
if "Year" in df_raw.columns: 
        df_raw["Year"] = pd.to_numeric(df_raw["Year"], errors="coerce").astype("Int64")

if "Production Tonnes" in df_raw.columns: 
        df_raw["Production Tonnes"] = pd.to_numeric(df_raw["Production Tonnes"], errors="coerce")

df_raw.dtypes

Domain Code           object
Domain                object
Area Code (M49)        int64
Country               object
Element Code           int64
Element               object
Item Code (CPC)        int64
Item                  object
Year Code              int64
Year                   Int64
Unit                  object
Production Tonnes    float64
Flag                  object
Flag Description      object
Note                  object
Forest_area_pct      float64
dtype: object

In [6]:
df_raw.shape

(50, 16)

In [7]:
if "Year" in df_raw.columns and "Production Tonnes" in df_raw.columns:
    latest = int(df_raw["Year"].dropna().max())
    df_raw[df_raw["Year"].eq(latest)].nlargest(10, "Production Tonnes")

In [8]:
out_path = "data/FAOSTAT_data_en_9-14-2025.csv"
df_raw.to_csv(out_path, index=False)
out_path

'data/FAOSTAT_data_en_9-14-2025.csv'

In [9]:
import sys 
!{sys.executable} -m pip install pandas requests tqdm

In [10]:
import pandas as pd
import requests
countries = {
    "Indonesia": "IDN",
    "Malaysia": "MYS", 
    "Thailand": "THA", 
    "Colombia": "COL", 
    "Nigeria": "NGA"
}

indicator = "AG.LND.FRST.ZS"
codes_str = ";".join(countries.values())
url = f"https://api.worldbank.org/v2/country/{codes_str}/indicator/{indicator}"
params = {"format": "json", "per_page": 20000}

In [11]:
r = requests.get(url, params=params, timeout=60)
r.raise_for_status()
data = r.json()

rows = data[1] if isinstance(data, list) and len(data) > 1 else []
len(rows), type(rows)

(325, list)

In [12]:
df_wb = pd.DataFrame([{
    "Country": rec["country"]["value"], 
    "Year": int(rec["date"]), 
    "Forest_area_pct": rec["value"]
} for rec in rows if rec.get("date") and rec.get("country")])

df_wb["Forest_area_pct"] = pd.to_numeric(df_wb["Forest_area_pct"], errors="coerce")
df_wb = df_wb.sort_values(["Country", "Year"]).reset_index(drop=True)

df_wb.head()

,Country,Year,Forest_area_pct
0,Colombia,1960,NaN
1,Colombia,1961,NaN
2,Colombia,1962,NaN
3,Colombia,1963,NaN
4,Colombia,1964,NaN


In [13]:
df_palm = pd.read_csv("data/FAOSTAT_data_en_9-14-2025.csv")
df_palm = df_palm[df_palm["Country"].isin(countries.keys())].copy()
df_join = pd.merge(df_palm, df_wb, on=["Country", "Year"], how="inner")
df_join.head()

,Domain Code,Domain,Area Code (M49),Country,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Production Tonnes,Flag,Flag Description,Note,Forest_area_pct_x,Forest_area_pct_y
0,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2013,2013,t,1040835.0,A,Official figure,NaN,54.442462,54.442462
1,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2014,2014,t,1109586.0,A,Official figure,NaN,54.321123,54.321123
2,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2015,2015,t,1275000.0,A,Official figure,NaN,54.199784,54.199784
3,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2016,2016,t,1146000.0,A,Official figure,NaN,54.041902,54.041902
4,QCL,Crops and livestock products,170,Colombia,5510,Production,2165,Palm oil,2017,2017,t,1627552.0,A,Official figure,NaN,53.843867,53.843867


In [14]:
df_wb.to_csv("data/forest_area_pct_worldbank.csv", index=False)
df_join.to_csv("data/FAOSTAT_data_en_9-14-2025.csv", index=False)

In [20]:
keep_cols = ["Country", "Year", "Production Tonnes", "Forest_area_pct_x"]
df_clean = df_join[keep_cols].copy()

out_path = "data/palm_vs_forest_clean.csv"
df_clean.to_csv(out_path, index=False)

df_clean.head(), out_path

(    Country  Year  Production Tonnes  Forest_area_pct_x
 0  Colombia  2013          1040835.0          54.442462
 1  Colombia  2014          1109586.0          54.321123
 2  Colombia  2015          1275000.0          54.199784
 3  Colombia  2016          1146000.0          54.041902
 4  Colombia  2017          1627552.0          53.843867,
 'data/palm_vs_forest_clean.csv')

In [16]:
import pandas as pd 
file_path = "data/cosmetics.csv"
df_cosmetics = pd.read_csv(file_path)

In [17]:
df_cosmetics.head()

,Unnamed: 0,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,palm_matches,has_palm
0,0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1,[],False
1,1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1,[],False
2,2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0,[],False
3,3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1,['Glyceryl Stearate'],True
4,4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1,['Stearic Acid'],True


In [18]:
import re

palm_patterns = [
    r"palm oil", 
    r"palm kernel", 
    r"sodium palmate", 
    r"sodium palm kernelate", 
    r"elaeis guineensis", 
    r"cetyl palmitate", 
    r"ascorbyl palmitate", 
    r"ethylexyl palmitate", 
    r"isopropyl palmitate", 
    r"stearic acid", 
    r"glyceryl stearate"
]

palm_regex = re.compile("|".join([re.escape(term) for term in palm_patterns]), flags=re.IGNORECASE)
df_cosmetics["has_palm"] = df_cosmetics["Ingredients"].apply(
    lambda x: bool(palm_regex.search(str(x)))
)

df_cosmetics["has_palm"].value_counts()

has_palm
False    973
True     499
Name: count, dtype: int64

In [19]:
def find_palm_matches(ingredients):
    if not isinstance(ingredients, str):
        return []
    return palm_regex.findall(ingredients)

df_cosmetics["palm_matches"] = df_cosmetics["Ingredients"].apply(find_palm_matches)
df_cosmetics["has_palm"] = df_cosmetics["palm_matches"].apply(lambda x: len(x) > 0)
df_cosmetics[["Brand", "Name", "palm_matches", "has_palm"]].head(10)
df_cosmetics.to_csv("data/cosmetics.csv")

In [61]:
from collections import Counter
all_matches = [m.lower() for matches in df_cosmetics["palm_matches"] for m in matches]
Counter(all_matches).most_common(10)

[('glyceryl stearate', 315),
 ('stearic acid', 253),
 ('ascorbyl palmitate', 82),
 ('elaeis guineensis', 28),
 ('isopropyl palmitate', 24),
 ('cetyl palmitate', 23),
 ('palm kernel', 7),
 ('palm oil', 5),
 ('sodium palmate', 1)]

In [63]:
df_palm_products = df_cosmetics[df_cosmetics["has_palm"]].copy()
df_palm_products[["Brand", "Name", "Price", "Ingredients", "palm_matches"]].head(10)

,Brand,Name,Price,Ingredients,palm_matches
3,LA MER,The Moisturizing Soft Cream,175,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",[Glyceryl Stearate]
4,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,"Water, Snail Secretion Filtrate, Phenyl Trimet...",[Stearic Acid]
6,DRUNK ELEPHANT,Lala Retro™ Whipped Cream,60,"Water, Glycerin, Caprylic/ Capric Triglyceride...","[Stearic Acid, Glyceryl Stearate]"
8,KIEHL'S SINCE 1851,Ultra Facial Cream,29,"Water, Glycerin, Cyclohexasiloxane, Squalane, ...","[Glyceryl Stearate, Stearic Acid]"
12,BELIF,The True Cream Aqua Bomb,38,"Water, Dipropylene Glycol, Glycerin, Methl Tri...","[Stearic Acid, Glyceryl Stearate]"
15,DRUNK ELEPHANT,The Littles™,90,"Beste™ No.9 Jelly Cleanser: Water, Sodium Laur...","[Stearic Acid, Glyceryl Stearate, stearic Acid]"
16,FIRST AID BEAUTY,Ultra Repair® Cream Intense Hydration,30,"Water, Stearic Acid, Glycerin, C12-15 Alkyl Be...","[Stearic Acid, Glyceryl Stearate, Glyceryl Ste..."
19,SK-II,R.N.A. POWER Face Cream,230,"Water, Glycerin, Galactomyces Ferment Filtrate...",[Stearic Acid]
21,BAREMINERALS,COMPLEXION RESCUE™ Tinted Moisturizer Broad Sp...,30,"Water, Coconut Alkanes, Propanediol, Squalane,...",[stearic Acid]
23,FRESH,Black Tea Firming Overnight Mask,92,"Water, Glycerin, Butylene Glycol, Jojoba Ester...",[Glyceryl Stearate]


In [64]:
df_palm_products.to_csv("data/cosmetics_with_palm.csv")

In [38]:
df = pd.read_csv("data/cosmetics_with_palm.csv")

In [65]:
import ast

if "palm_matches" in df.columns: 
    def safe_parse(val):
        if isinstance(val, str) and val.startswith("["):
            try: 
                return ast.literal_eval(val)
            except:
                return []
        return val if isinstance(val, list) else []
    df["palm_matches"] = df["palm_matches"].apply(safe_parse)

    df_palm_clean = df[["Brand", "Name", "Ingredients", "has_palm", "palm_matches"]].copy()
    df_palm_clean.head()

In [66]:
summary_by_label = (
    df_cosmetics.groupby(df_cosmetics["Label"].astype(str).str.strip(), dropna=False)
        .agg(total_products=("Label", "size"), 
             palm_products=("has_palm", "sum"))
        .reset_index()
        .rename(columns={"Label":"Product_Type"})
)

summary_by_label["palm_share_pct"] = (
    summary_by_label["palm_products"] / summary_by_label["total_products"] * 100
).round(1)

out_path = "data/product_palm_summary.csv"
summary_by_label.to_csv(out_path, index=False)

In [53]:
out_path = "data/cosmetics_palm_only.csv"
df_palm_clean.to_csv(out_path, index=False)
out_path

'data/cosmetics_palm_only.csv'

In [68]:
def unique_matches_list(series):
    acc = set()
    for lst in series: 
        if isinstance(lst, list):
            for item in lst:
                if isinstance(item, str):
                    acc.add(item.strip().lower())
    return "; ".join(sorted(acc)) if acc else ""

summary = (df.assign(Brand=df["Brand"].astype(str).str.strip())
           .groupby("Brand", dropna=False)
           .agg(palm_products=("has_palm", "sum"), 
                palm_terms=("palm_matches", unique_matches_list))
           .reset_index())

summary = summary.sort_values(["palm_products"], ascending=[False]).reset_index(drop=True)
summary.head(20)

out_path = "data/brand_summary.csv"
summary.to_csv(out_path, index=False)